In [1]:
%%capture
from pathlib import Path

if Path.cwd().stem == "features":
    %cd ../..
    %load_ext autoreload
    %autoreload 2

In [2]:
import logging
from pathlib import Path

import holoviews as hv
import plotly.io as pio
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.features.scaling import scale_min_max
from src.features.transformations import (
    add_time_column,
)
from src.helpers import to_describe
from src.log_config import configure_logging

configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

polars.config.Config

In [3]:
db = DatabaseManager()

In [4]:
with db:
    df = db.read_table("Raw_Stimulus")
df

trial_id,trial_number,participant_id,rownumber,timestamp,temperature,rating
u16,u8,u8,u32,f64,f64,f64
1,1,1,0,294224.331,45.75,42.5
1,1,1,1,294357.9645,45.750102,42.5
1,1,1,2,294458.0292,45.75041,35.375
1,1,1,3,294558.6006,45.750922,14.875
1,1,1,4,294658.3354,45.751639,10.125
1,1,1,5,294758.4957,45.75256,22.75
…,…,…,…,…,…,…
332,12,28,21606,2.7766e6,45.587356,85.0
332,12,28,21607,2.7767e6,45.585285,85.0
